# Generalizations of Max-Flow

The purpose of this assignment is to learn about the min-cost flow problem, a generalization of max-flow, and to familiarize yourself with implementing and solving linear programs.

## Min-Cost Flow

Recall that a flow network with demands consists of a directed graph $G = (V, E)$, where each edge $(i,j) \in E$ has a positive integer capacity $c_{ij}$ and each node $i \in V$ has an integer demand $d_i$. In a *min-cost flow* problem, each edge $(i,j) \in E$ also has a cost (or weight) $w_{ij}$. (Note that this input generalizes the input to two important problems we discussed so far: in max flow, the edge weights were not important while in shortest paths, the edge capacities were not important.) 

Given a flow network with capacities and costs, the goal is to find a *feasible* flow $f: E \rightarrow R^+$ --that is, a flow satisfying edge capacity constraints and node demands-- that minimizes the total cost of the flow. Explicitly, the problem can be formulated as a linear program.

### Question 1

Answer Problem 1 in HW4-theoretical.

### Question 2

To implement your reduction from Problem 1 in HW4-theoretical, you will work with some standard benchmark instances for min-cost flow found [here](http://elib.zib.de/pub/Packages/mp-testdata/mincost/gte/index.html). The format of the data is described in the [Info](http://elib.zib.de/pub/Packages/mp-testdata/mincost/gte/info) file. You are to read in the graph from the data file in a form that can be solved using NetworkX's `min_cost_flow` function. Note that the data sometimes lists multiple edges between the same nodes, but with different costs or capacities. In forming the graph, you need to implement your reduction from the previous question and form a `DiGraph` instance, because the `min_cost_flow` function cannot handle multi-edges, even though the package offers `MultiDiGraph` objects.

In [9]:
import networkx as nx

def create_graph(infile):
    """Creates a directed graph as specified by the input file. Edges are annotated with 'capacity'
    and 'weight' attributes, and nodes are annotated with 'demand' attributes.
    
    Args:
        infile: the input file using the format to specify a min-cost flow problem.
        
    Returns:
        A directed graph (but not a multi-graph) with edges annotated with 'capacity' and 'weight' attributes
        and nodes annotated with 'demand' attributes.
    """
    # TODO: implement function
    
    with open(infile) as f:
        content = f.read().splitlines()
    
    G = nx.DiGraph()
    numNodes = int(content[1].split()[2])

    for node in range(1,numNodes+1):
        G.add_node(node)

    newNode = numNodes

    for line in range(0,len(content)-1):
        if content[line].split()[0] == 'n':
            node = int(content[line].split()[1])
            G.node[node]['demand'] = int(content[line].split()[2])

        if content[line].split()[0] == 'a':
            u = int(content[line].split()[1])
            v = int(content[line].split()[2])
            cap = int(content[line].split()[4])
            thecost = int(content[line].split()[5])

            if (u,v) in G.edges():
                newNode += 1
                G.add_node(newNode)
                G.add_edge(u, newNode, capacity = cap, weight=thecost)
                G.add_edge(newNode, v, capacity = cap, weight=0)
            else:
                G.add_edge(u, v, capacity = cap, weight=thecost)
    
    return(G)

The following will check that your code outputs the expected min cost flow values on several test instances.

In [10]:
G_40 = create_graph('gte_bad.40')
G_6830 = create_graph('gte_bad.6830')
G_176280 = create_graph('gte_bad.176280')

print "Correct value for _40 instance:", nx.min_cost_flow_cost(G_40) == 52099553858
print "Correct value for _6830 instance:", nx.min_cost_flow_cost(G_6830) == 299390431788
print "Correct value for _176280 instance:", nx.min_cost_flow_cost(G_176280) == 510585093810

Correct value for _40 instance: True
Correct value for _6830 instance: True
Correct value for _176280 instance: True


## Linear Programming

Instead of using special-purpose min-cost flow solvers, you will now formulate the problems as linear programs and use general-purpose LP solvers to find the solutions.

### Question 3

Implement the following function to formulate the flow LP and return the optimal value (i.e., minimum cost over feasible flows).

In [11]:
import pulp

def lp_flow_value(G):
    """Computes the value of the minimum cost flow by formulating and solving
    the problem as an LP.
    
    Args:
        G: a directed graph with edges annotated with 'capacity' and 'weight'
            attrbutes, and nodes annotated with 'demand' attributes.
            
    Returns:
        The value of the minimum cost flow.
    """
    # TODO: implement function
    
    prob = pulp.LpProblem("minCostFlow", pulp.LpMinimize)
    
    goal_func = []
    
    for node in G.nodes():
        if G.node[node] == {}:
            G.node[node]['demand'] = 0
            G.node[node]['flowin'] = []
            G.node[node]['flowout'] = []
        else:
            G.node[node]['flowin'] = []
            G.node[node]['flowout'] = []
    
    for node in G.nodes():
        for conn in G[node]:
            flow = pulp.LpVariable("x_" + str(node) + "_" + str(conn),lowBound=0, upBound=G[node][conn]['capacity'])
            weight = G[node][conn]['weight']
            
            goal_func.append(weight*flow)
            
            G.node[node]['flowin'].append(flow)
            G.node[conn]['flowout'].append(flow)
        
    for node in G.nodes():
        prob += sum(G.node[node]['flowin']) - sum(G.node[node]['flowout']) == G.node[node]['demand']

    prob += sum(goal_func)
    
    prob.solve()
    
    return pulp.value(pulp.lpSum(goal_func))

The following will check that the LP finds the same optimal values as previously.

In [12]:
print "Correct value for _40 instance:", lp_flow_value(G_40) == 52099553858
print "Correct value for _6830 instance:", lp_flow_value(G_6830) == 299390431788
print "Correct value for _176280 instance:", lp_flow_value(G_176280) == 510585093810

Correct value for _40 instance: True
Correct value for _6830 instance: True
Correct value for _176280 instance: True
